In [7]:
import sys
import matplotlib.pyplot as plt
import numpy as np
# if using a Jupyter notebook, includue:
%matplotlib inline
import time
import keras
from enum import Enum
import keras
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras import layers
from math import floor, ceil
from random import sample
import pickle

In [8]:
# TODO: currently hard-coded maxlen to 10...
# TODO: currently won't work unless there are exactly 10 words
def predictNext(model, primer):
    prime = primer.split()
    if (len(prime) > 10):
        return False
    # Will feed to batch prediction, with shape (None, maxlen)
    arr = np.zeros((1,10), dtype=np.int)
    for i in range(0,10):
        if prime[i] in model.word_index2:
            arr[0][i] = model.word_index2[prime[i]]
        else:
            arr[0][i] = len(model.word_index2)
    print(arr)
    return model.model.predict_on_batch(arr)
def maxVal(arr):
    tmp = 0
    for i in range(1,len(arr)):
        if arr[i] > arr[tmp]:
            tmp = i
    return tmp
def findWordFromIndexSlowly(model, idx):
    for key, value in model.word_index2.items():
        if value == idx:
            return key
    return False
def predictNextSplit(model, prime):
    # Will feed to batch prediction, with shape (None, maxlen)
    arr = np.zeros((1,model.maxlen), dtype=np.int)
    for i in range(0,model.maxlen):
        if prime[i] in model.word_index2:
            arr[0][i] = model.word_index2[prime[i]]
        else:
            arr[0][i] = len(model.word_index2)
    return model.model.predict_on_batch(arr)[0]
def next5(model, primer):
    prime = primer.split()
    if (len(prime) > model.maxlen):
        prime = prime[-10:]
    elif len(prime) < model.maxlen:
        prime = (("UNDEF "*(model.maxlen-len(prime)))+primer).split()
    orig = list()
    for i in range(0,5):
        w = findWordFromIndexSlowly(model, maxVal(predictNextSplit(model, prime)))
        prime.append(w)
        orig.append(prime.pop(0))
    orig.extend(prime)
    while (orig[0] == 'UNDEF'):
        orig.remove('UNDEF')
    return ' '.join(orig)

In [11]:
class PredictionsOnly:
    def __init__(self, name, d, m):
        self.word_index2 = d
        self.model = m
        self.name = name
        self.maxlen = 10

def loadModel(prefix):
    d = None
    with open(prefix+"_shortgru.dict", "rb") as handle:
        d = pickle.load(handle)
    m = keras.models.load_model(prefix+"_shortgru.h5") #, save_format='h5')
    return PredictionsOnly(prefix, d, m)

In [14]:
models = [loadModel('Advanced_Python'), loadModel('Raw_Python')] #, loadModel('Basic_Python'), loadModel('Raw_Python')]

In [15]:
def predictAll(seed):
    for m in models:
        print(m.name+": "+next5(m, seed))

In [16]:
predictAll("throws an exception if")

Advanced_Python: throws an exception if the user is not a
Raw_Python: throws an exception if the exception is raised if


In [17]:
predictAll("throws an exception when")

Advanced_Python: throws an exception when the exception is raised if
Raw_Python: throws an exception when the user is not a


In [18]:
predictAll("raises an exception if")

Advanced_Python: raises an exception if the user is not a
Raw_Python: raises an exception if the exception is raised if


In [26]:
# gen some fresh ideal samples
#import random
#with open("commentdata2/advanced","r") as f:
#    comments = f.readlines()
#sample = random.sample(list(filter(lambda l: len(l) > 3, map(lambda s: s.split(' '),comments))), 1000)
#sample = list(map(lambda l: l[0:4], sample))

In [32]:
#len(sample)

1000

In [35]:
#with open("advPhrases","w") as f:
#    for s in sample:
#        f.write(str(' '.join(s))+"\n")
#        f.flush()

In [38]:
with open("advPhrases","r") as p:
    phrases = p.readlines()

for m in models:
    prefix = m.name.replace(' ','_')
    with open(prefix+"_genadv.txt","w") as out:
        for p in phrases:
            out.write(next5(m,p)+"\n")